### In this notebook we perform federated learning

In federated learning each base station has access only to it's private dataset, however they collaborate together to train a model that has satifactory results on data from any other base station.

In [1]:
import sys
import os

from pathlib import Path

parent = Path(os.path.abspath("")).resolve().parents[0]
if parent not in sys.path:
    sys.path.insert(0, str(parent))

In [2]:
import copy

import random

from collections import OrderedDict

import numpy as np
import torch

from argparse import Namespace

In [3]:
from ml.utils.data_utils import read_data, generate_time_lags, time_to_feature, handle_nans, to_Xy, \
    to_torch_dataset, to_timeseries_rep, assign_statistics, \
    to_train_val, scale_features, get_data_by_area, remove_identifiers, get_exogenous_data_by_area, handle_outliers

In [4]:
from ml.utils.train_utils import train, test

In [5]:
from ml.models.mlp import MLP
from ml.models.rnn import RNN
from ml.models.lstm import LSTM
from ml.models.gru import GRU
from ml.models.cnn import CNN
from ml.models.rnn_autoencoder import DualAttentionAutoEncoder

In [6]:
from ml.fl.defaults import create_regression_client
from ml.fl.client_proxy import SimpleClientProxy
from ml.fl.server.server import Server

In [7]:
args = Namespace(
    data_path='../dataset/full_dataset.csv', # dataset
    data_path_test=['../dataset/ElBorn_test.csv'], # test dataset
    test_size=0.2, # validation size 
    targets=['rnti_count', 'rb_down', 'rb_up', 'down', 'up'], # the target columns
    num_lags=10, # the number of past observations to feed as input

    identifier='District', # the column name that identifies a bs

    nan_constant=0, # the constant to transform nan values
    x_scaler='minmax', # x_scaler
    y_scaler='minmax', # y_scaler
    outlier_detection=None, # whether to perform flooring and capping

    
    criterion='mse', # optimization criterion, mse or l1
    fl_rounds=10, # the number of federated rounds
    fraction=1., # the percentage of available client to consider for random selection
    aggregation="fedavg", # federated aggregation algorithm
    epochs=3, # the number of maximum local epochs
    lr=0.001, # learning rate
    optimizer='adam', # the optimizer, it can be sgd or adam
    batch_size=128, # the batch size to use
    local_early_stopping=False, # whether to use early stopping
    local_patience=50, # patience value for the early stopping parameter (if specified)
    max_grad_norm=0.0, # whether to clip grad norm
    reg1=0.0, # l1 regularization
    reg2=0.0, # l2 regularization

    cuda=True, # whether to use gpu
    
    seed=0, # reproducibility

    assign_stats=None, # whether to use statistics as exogenous data, ["mean", "median", "std", "variance", "kurtosis", "skew"]
    use_time_features=False # whether to use datetime features
)

In [8]:
print(f"Script arguments: {args}\n")

Script arguments: Namespace(data_path='../dataset/full_dataset.csv', data_path_test=['../dataset/ElBorn_test.csv'], test_size=0.2, targets=['rnti_count', 'rb_down', 'rb_up', 'down', 'up'], num_lags=10, identifier='District', nan_constant=0, x_scaler='minmax', y_scaler='minmax', outlier_detection=None, criterion='mse', fl_rounds=10, fraction=1.0, aggregation='fedavg', epochs=3, lr=0.001, optimizer='adam', batch_size=128, local_early_stopping=False, local_patience=50, max_grad_norm=0.0, reg1=0.0, reg2=0.0, cuda=True, seed=0, assign_stats=None, use_time_features=False)



In [9]:
device = "cuda" if args.cuda and torch.cuda.is_available() else "cpu"
print(f"Using {device}")

Using cuda


In [10]:
# Outlier detection specification
if args.outlier_detection is not None:
    outlier_columns = ['rb_down', 'rb_up', 'down', 'up']
    outlier_kwargs = {"ElBorn": (10, 90), "LesCorts": (10, 90), "PobleSec": (5, 95)}
    args.outlier_columns = outlier_columns
    args.outlier_kwargs = outlier_kwargs

In [11]:
def seed_all():
    # ensure reproducibility
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [12]:
seed_all()

### The pre-processing method is almost equivalent to centralized learning. The only difference is that the scaling operations are performed individually on each base station. In contrast, in centralized learning the scaling is performed by considering the combined data from all base stations.

In [13]:
def make_preprocessing():
    """Preprocess a given .csv"""
    # read data
    df = read_data(args.data_path)
    # handle nans
    df = handle_nans(train_data=df, constant=args.nan_constant,
                     identifier=args.identifier)
    # split to train/validation
    train_data, val_data = to_train_val(df)
    
    # handle outliers (if specified)
    if args.outlier_detection is not None:
        train_data = handle_outliers(df=train_data, columns=args.outlier_columns,
                                     identifier=args.identifier, kwargs=args.outlier_kwargs)
    
    # get X and y
    X_train, X_val, y_train, y_val = to_Xy(train_data=train_data, val_data=val_data,
                                          targets=args.targets)
    
    # scale X
    X_train, X_val, x_scalers = scale_features(train_data=X_train, val_data=X_val,
                                              scaler=args.x_scaler,
                                              per_area=True, # the features are scaled locally
                                              identifier=args.identifier)
    # scale y
    y_train, y_val, y_scalers = scale_features(train_data=y_train, val_data=y_val,
                                              scaler=args.y_scaler, 
                                              per_area=True,
                                              identifier=args.identifier)
    
    # generate time lags
    X_train = generate_time_lags(X_train, args.num_lags)
    X_val = generate_time_lags(X_val, args.num_lags)
    y_train = generate_time_lags(y_train, args.num_lags, is_y=True)
    y_val = generate_time_lags(y_val, args.num_lags, is_y=True)
    
    # get datetime features as exogenous data
    date_time_df_train = time_to_feature(
        X_train, args.use_time_features, identifier=args.identifier
    )
    date_time_df_val = time_to_feature(
        X_val, args.use_time_features, identifier=args.identifier
    )
    
    # get statistics as exogenous data
    stats_df_train = assign_statistics(X_train, args.assign_stats, args.num_lags,
                                       targets=args.targets, identifier=args.identifier)
    stats_df_val = assign_statistics(X_val, args.assign_stats, args.num_lags, 
                                       targets=args.targets, identifier=args.identifier)
    
    # concat the exogenous features (if any) to a single dataframe
    if date_time_df_train is not None or stats_df_train is not None:
        exogenous_data_train = pd.concat([date_time_df_train, stats_df_train], axis=1)
        # remove duplicate columns (if any)
        exogenous_data_train = exogenous_data_train.loc[:, ~exogenous_data_train.columns.duplicated()].copy()
        assert len(exogenous_data_train) == len(X_train) == len(y_train)
    else:
        exogenous_data_train = None
    if date_time_df_val is not None or stats_df_val is not None:
        exogenous_data_val = pd.concat([date_time_df_val, stats_df_val], axis=1)
        exogenous_data_val = exogenous_data_val.loc[:, ~exogenous_data_val.columns.duplicated()].copy()
        assert len(exogenous_data_val) == len(X_val) == len(y_val)
    else:
        exogenous_data_val = None
        
    return X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scalers, y_scalers

In [14]:
X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scalers, y_scalers = make_preprocessing()

INFO logger 2022-10-24 23:13:48,390 | data_utils.py:383 | Observations info in ElBorn
INFO logger 2022-10-24 23:13:48,390 | data_utils.py:384 | 	Total number of samples:  4192
INFO logger 2022-10-24 23:13:48,390 | data_utils.py:385 | 	Number of samples for training: 3354
INFO logger 2022-10-24 23:13:48,390 | data_utils.py:386 | 	Number of samples for validation:  838
INFO logger 2022-10-24 23:13:48,398 | data_utils.py:383 | Observations info in LesCorts
INFO logger 2022-10-24 23:13:48,398 | data_utils.py:384 | 	Total number of samples:  6892
INFO logger 2022-10-24 23:13:48,398 | data_utils.py:385 | 	Number of samples for training: 5514
INFO logger 2022-10-24 23:13:48,398 | data_utils.py:386 | 	Number of samples for validation:  1378
INFO logger 2022-10-24 23:13:48,406 | data_utils.py:383 | Observations info in PobleSec
INFO logger 2022-10-24 23:13:48,406 | data_utils.py:384 | 	Total number of samples:  15927
INFO logger 2022-10-24 23:13:48,406 | data_utils.py:385 | 	Number of samples f

In [15]:
X_train.head()

,rb_up_var_lag-10,rb_up_lag-10,rb_down_var_lag-10,rb_down_lag-10,mcs_up_var_lag-10,mcs_up_lag-10,mcs_down_var_lag-10,mcs_down_lag-10,rnti_count_lag-10,up_lag-10,...,rb_down_var_lag-1,rb_down_lag-1,mcs_up_var_lag-1,mcs_up_lag-1,mcs_down_var_lag-1,mcs_down_lag-1,rnti_count_lag-1,up_lag-1,down_lag-1,District
time,,,,,,,,,,,,,,,,,,,,,
2018-03-28 16:16:00,3.143298e-08,0.002418,2.677239e-08,0.043020,0.207425,0.483274,0.796265,0.929664,0.227829,0.002758,...,2.890976e-08,0.052571,0.232918,0.478785,0.793296,0.916750,0.275640,0.003258,0.108785,ElBorn
2018-03-28 16:18:00,4.439640e-08,0.003812,2.795076e-08,0.052412,0.259314,0.530084,0.796778,0.914716,0.273796,0.004257,...,2.742117e-08,0.052254,0.242482,0.499756,0.783107,0.922241,0.274142,0.003790,0.108356,ElBorn
2018-03-28 16:20:00,2.993595e-08,0.002716,2.825645e-08,0.047602,0.261772,0.512427,0.797310,0.921577,0.249107,0.002874,...,2.813661e-08,0.051305,0.241381,0.450879,0.800429,0.920308,0.269116,0.002989,0.106185,ElBorn
2018-03-28 16:22:00,5.382563e-08,0.004139,2.711694e-08,0.060476,0.320280,0.506925,0.782003,0.916003,0.315683,0.004442,...,2.869276e-08,0.061644,0.315197,0.495057,0.814955,0.917776,0.317020,0.004932,0.127269,ElBorn
2018-03-28 16:24:00,5.922178e-08,0.004565,2.835084e-08,0.066004,0.286799,0.497228,0.781283,0.919718,0.343507,0.004884,...,2.695933e-08,0.042399,0.267656,0.452835,0.792680,0.919174,0.226423,0.002596,0.088458,ElBorn


In [16]:
y_train.head()

,rnti_count,rb_down,rb_up,down,up,District
time,,,,,,
2018-03-28 16:16:00,0.274142,0.052254,0.003433,0.108356,0.003790,ElBorn
2018-03-28 16:18:00,0.269116,0.051305,0.002995,0.106185,0.002989,ElBorn
2018-03-28 16:20:00,0.317020,0.061644,0.004508,0.127269,0.004932,ElBorn
2018-03-28 16:22:00,0.226423,0.042399,0.002672,0.088458,0.002596,ElBorn
2018-03-28 16:24:00,0.307914,0.058735,0.003842,0.122086,0.004053,ElBorn


In [17]:
x_scalers, y_scalers

({'ElBorn': MinMaxScaler(),
  'LesCorts': MinMaxScaler(),
  'PobleSec': MinMaxScaler()},
 {'ElBorn': MinMaxScaler(),
  'LesCorts': MinMaxScaler(),
  'PobleSec': MinMaxScaler()})

### Postprocessing in a same manner with centalized learning

In [18]:
def make_postprocessing(X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scalers, y_scalers):
    """Make data ready to be fed into ml algorithms"""
    # if there are more than one specified areas, get the data per area
    if X_train[args.identifier].nunique() != 1:
        area_X_train, area_X_val, area_y_train, area_y_val = get_data_by_area(X_train, X_val,
                                                                              y_train, y_val, 
                                                                              identifier=args.identifier)
    else:
        area_X_train, area_X_val, area_y_train, area_y_val = None, None, None, None

    # Get the exogenous data per area.
    if exogenous_data_train is not None:
        exogenous_data_train, exogenous_data_val = get_exogenous_data_by_area(exogenous_data_train,
                                                                              exogenous_data_val)
    # transform to np
    if area_X_train is not None:
        for area in area_X_train:
            tmp_X_train, tmp_y_train, tmp_X_val, tmp_y_val = remove_identifiers(
                area_X_train[area], area_y_train[area], area_X_val[area], area_y_val[area])
            tmp_X_train, tmp_y_train = tmp_X_train.to_numpy(), tmp_y_train.to_numpy()
            tmp_X_val, tmp_y_val = tmp_X_val.to_numpy(), tmp_y_val.to_numpy()
            area_X_train[area] = tmp_X_train
            area_X_val[area] = tmp_X_val
            area_y_train[area] = tmp_y_train
            area_y_val[area] = tmp_y_val
    
    if exogenous_data_train is not None:
        for area in exogenous_data_train:
            exogenous_data_train[area] = exogenous_data_train[area].to_numpy()
            exogenous_data_val[area] = exogenous_data_val[area].to_numpy()
    
    # remove identifiers from features, targets
    X_train, y_train, X_val, y_val = remove_identifiers(X_train, y_train, X_val, y_val)
    assert len(X_train.columns) == len(X_val.columns)
    
    num_features = len(X_train.columns) // args.num_lags
    
    # to timeseries representation
    X_train = to_timeseries_rep(X_train.to_numpy(), num_lags=args.num_lags,
                                            num_features=num_features)
    X_val = to_timeseries_rep(X_val.to_numpy(), num_lags=args.num_lags,
                                          num_features=num_features)
    
    if area_X_train is not None:
        area_X_train = to_timeseries_rep(area_X_train, num_lags=args.num_lags,
                                                     num_features=num_features)
        area_X_val = to_timeseries_rep(area_X_val, num_lags=args.num_lags,
                                                   num_features=num_features)
    
    # transform targets to numpy
    y_train, y_val = y_train.to_numpy(), y_val.to_numpy()
    
    if exogenous_data_train is not None:
        exogenous_data_train_combined, exogenous_data_val_combined = [], []
        for area in exogenous_data_train:
            exogenous_data_train_combined.extend(exogenous_data_train[area])
            exogenous_data_val_combined.extend(exogenous_data_val[area])
        exogenous_data_train_combined = np.stack(exogenous_data_train_combined)
        exogenous_data_val_combined = np.stack(exogenous_data_val_combined)
        exogenous_data_train["all"] = exogenous_data_train_combined
        exogenous_data_val["all"] = exogenous_data_val_combined
    return X_train, X_val, y_train, y_val, area_X_train, area_X_val, area_y_train, area_y_val, exogenous_data_train, exogenous_data_val

In [19]:
X_train, X_val, y_train, y_val, client_X_train, client_X_val, client_y_train, client_y_val, exogenous_data_train, exogenous_data_val = make_postprocessing(X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scalers, y_scalers)

In [20]:
client_X_train.keys()

dict_keys(['ElBorn', 'LesCorts', 'PobleSec'])

In [21]:
client_X_val.keys()

dict_keys(['ElBorn', 'LesCorts', 'PobleSec'])

In [22]:
for client in client_X_train:
    print(f"\nClient: {client}")
    print(f"X_train shape: {client_X_train[client].shape}, y_train shape: {client_y_train[client].shape}")
    print(f"X_val shape: {client_X_val[client].shape}, y_val shape: {client_y_val[client].shape}")


Client: ElBorn
X_train shape: (3344, 10, 11, 1), y_train shape: (3344, 5)
X_val shape: (828, 10, 11, 1), y_val shape: (828, 5)

Client: LesCorts
X_train shape: (5504, 10, 11, 1), y_train shape: (5504, 5)
X_val shape: (1368, 10, 11, 1), y_val shape: (1368, 5)

Client: PobleSec
X_train shape: (12732, 10, 11, 1), y_train shape: (12732, 5)
X_val shape: (3175, 10, 11, 1), y_val shape: (3175, 5)


In [23]:
def get_input_dims(X_train, exogenous_data_train):
    if args.model_name == "mlp":
        input_dim = X_train.shape[1] * X_train.shape[2]
    else:
        input_dim = X_train.shape[2]
    
    if exogenous_data_train is not None:
        if len(exogenous_data_train) == 1:
            cid = next(iter(exogenous_data_train.keys()))
            exogenous_dim = exogenous_data_train[cid].shape[1]
        else:
            exogenous_dim = exogenous_data_train["all"].shape[1]
    else:
        exogenous_dim = 0
    
    return input_dim, exogenous_dim

In [24]:
def get_model(model: str,
              input_dim: int,
              out_dim: int,
              lags: int = 10,
              exogenous_dim: int = 0,
              seed=0):
    if model == "mlp":
        model = MLP(input_dim=input_dim, layer_units=[256, 128, 64], num_outputs=out_dim)
    elif model == "rnn":
        model = RNN(input_dim=input_dim, rnn_hidden_size=128, num_rnn_layers=1, rnn_dropout=0.0,
                    layer_units=[128], num_outputs=out_dim, matrix_rep=True, exogenous_dim=exogenous_dim)
    elif model == "lstm":
        model = LSTM(input_dim=input_dim, lstm_hidden_size=128, num_lstm_layers=1, lstm_dropout=0.0,
                     layer_units=[128], num_outputs=out_dim, matrix_rep=True, exogenous_dim=exogenous_dim)
    elif model == "gru":
        model = GRU(input_dim=input_dim, gru_hidden_size=128, num_gru_layers=1, gru_dropout=0.0,
                    layer_units=[128], num_outputs=out_dim, matrix_rep=True, exogenous_dim=exogenous_dim)
    elif model == "cnn":
        model = CNN(num_features=input_dim, lags=lags, exogenous_dim=exogenous_dim, out_dim=out_dim)
    elif model == "da_encoder_decoder":
        model = DualAttentionAutoEncoder(input_dim=input_dim, architecture="lstm", matrix_rep=True)
    else:
        raise NotImplementedError("Specified model is not implemented. Plese define your own model or choose one from ['mlp', 'rnn', 'lstm', 'gru', 'cnn', 'da_encoder_decoder']")
    return model

In [25]:
# define the model
args.model_name = "cnn"

input_dim, exogenous_dim = get_input_dims(X_train, exogenous_data_train)

print(input_dim, exogenous_dim)

model = get_model(model=args.model_name,
                  input_dim=input_dim,
                  out_dim=y_train.shape[1],
                  lags=args.num_lags,
                  exogenous_dim=exogenous_dim,
                  seed=args.seed)

11 0


In [26]:
model

CNN(
  (activation): ReLU()
  (conv1): Conv2d(1, 16, kernel_size=(16, 3), stride=(1, 1), padding=same)
  (conv2): Conv2d(16, 16, kernel_size=(3, 5), stride=(1, 1), padding=same)
  (conv3): Conv2d(16, 32, kernel_size=(4, 3), stride=(1, 1), padding=same)
  (conv4): Conv2d(32, 32, kernel_size=(4, 3), stride=(1, 1), padding=same)
  (pool): AvgPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0)
  (fc): Linear(in_features=1760, out_features=5, bias=True)
)

### Fit function initiates the training process of every base station local model and then performs parameters aggregation on a central server for N specified federated epochs.

In [27]:
def fit(model, X_train, y_train, X_val, y_val, 
        exogenous_data_train=None, exogenous_data_val=None, 
        idxs=[8, 3, 1, 10, 9], # the indices of our targets in X
        log_per=1,
        client_creation_fn = None, # client specification
        local_train_params=None, # local params
        aggregation_params=None, # aggregation params
        use_carbontracker=True
       ):
    # client creation definition
    if client_creation_fn is None:
        client_creation_fn = create_regression_client
    # local params
    if local_train_params is None:
        local_train_params = {
            "epochs": args.epochs, "optimizer": args.optimizer, "lr": args.lr,
            "criterion": args.criterion, "early_stopping": args.local_early_stopping,
            "patience": args.local_patience, "device": device
        }
    
    train_loaders, val_loaders = [], []
    
    # get data per client
    for client in X_train:
        if client == "all":
            continue
        if exogenous_data_train is not None:
            tmp_exogenous_data_train = exogenous_data_train[client]
            tmp_exogenous_data_val = tmp_exogenous_data_val[client]
        else:
            tmp_exogenous_data_train = None
            tmp_exogenous_data_val = None
    
        num_features = len(X_train[client][0][0])
        
        # to torch loader
        train_loaders.append(
            to_torch_dataset(
                X_train[client], y_train[client],
                num_lags=args.num_lags,
                num_features=num_features,
                exogenous_data=tmp_exogenous_data_train,
                indices=idxs,
                batch_size=args.batch_size,
                shuffle=False
            )
        )
        val_loaders.append(
            to_torch_dataset(
                X_val[client], y_val[client],
                num_lags=args.num_lags,
                exogenous_data=tmp_exogenous_data_val,
                indices=idxs,
                batch_size=args.batch_size,
                shuffle=False
            )
            
        )
        
    # create clients with their local data
    cids = [k for k in X_train.keys() if k != "all"]
    clients = [
        client_creation_fn(
            cid=cid, # client id
            model=model, # the global model
            train_loader=train_loader, # the local train loader
            test_loader=val_loader, # the local val loader
            local_params=local_train_params # local parameters
        )
        for cid, train_loader, val_loader in zip(cids, train_loaders, val_loaders)
    ]
    
    # represent clients to server
    client_proxies = [
        SimpleClientProxy(cid, client) for cid, client in zip(cids, clients)
    ]
    
    # represent the server
    server = Server(
        client_proxies=client_proxies, # the client representations
        aggregation=args.aggregation, # the aggregation algorithm
        aggregation_params=aggregation_params, # aggregation specific params
        local_params_fn=None, # we can change the local params on demand
    )
    # Note that the client manager instance will be initialized automatically. You can define your own client manager.

    # train with FL
    model_params, history = server.fit(args.fl_rounds, args.fraction, use_carbontracker=use_carbontracker)
    
    params_dict = zip(model.state_dict().keys(), model_params)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    model = copy.deepcopy(model)
    model.load_state_dict(state_dict, strict=True)
    
    return model, history

In [28]:
# federated local params
local_train_params = {"epochs": args.epochs, "optimizer": args.optimizer, "lr": args.lr,
                      "criterion": args.criterion, "early_stopping": args.local_early_stopping,
                      "patience": args.local_patience, "device": device
                      }

In [29]:
global_model, history = fit(
    model,
    client_X_train,
    client_y_train, 
    client_X_val, 
    client_y_val, 
    local_train_params=local_train_params
)

INFO logger 2022-10-24 23:13:52,924 | server.py:62 | Initializing client manager...
INFO logger 2022-10-24 23:13:52,924 | server.py:69 | Registering clients...
INFO logger 2022-10-24 23:13:52,924 | client_manager.py:66 | Registered client with id: ElBorn
INFO logger 2022-10-24 23:13:52,924 | client_manager.py:66 | Registered client with id: LesCorts
INFO logger 2022-10-24 23:13:52,924 | client_manager.py:66 | Registered client with id: PobleSec
INFO logger 2022-10-24 23:13:52,932 | server.py:73 | Client manager initialized!
INFO logger 2022-10-24 23:13:52,932 | server.py:55 | Aggregation algorithm: FedAvg()
INFO logger 2022-10-24 23:13:52,932 | client_manager.py:88 | Parameter c=0.0. Sampled 1 client(s): ['LesCorts']
c:\Users\nikop\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\conv.py:442: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ..\aten\src\AT

CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce GTX 1650 Ti.


INFO logger 2022-10-24 23:14:00,201 | train_utils.py:142 | Best Loss: 1.638515244615237e-05
INFO logger 2022-10-24 23:14:02,399 | train_utils.py:142 | Best Loss: 3.2951381799297022e-06
INFO logger 2022-10-24 23:14:06,641 | train_utils.py:142 | Best Loss: 1.6769740646191704e-05
INFO logger 2022-10-24 23:14:07,102 | server.py:191 | [Global round 1] Aggregating local models...


CarbonTracker: 
Actual consumption for 1 epoch(s):
	Time:	0:00:09
	Energy:	0.000055 kWh
	CO2eq:	0.016141 g
	This is equivalent to:
	0.000134 km travelled by car
CarbonTracker: 
Predicted consumption for 10 epoch(s):
	Time:	0:01:33
	Energy:	0.000549 kWh
	CO2eq:	0.161412 g
	This is equivalent to:
	0.001341 km travelled by car
CarbonTracker: Finished monitoring.


INFO logger 2022-10-24 23:14:09,559 | client_manager.py:88 | Parameter c=1.0. Sampled 3 client(s): ['PobleSec', 'LesCorts', 'ElBorn']
INFO logger 2022-10-24 23:14:16,032 | train_utils.py:142 | Best Loss: 1.580404676863645e-05
INFO logger 2022-10-24 23:14:18,205 | train_utils.py:142 | Best Loss: 1.4189285691997734e-05
INFO logger 2022-10-24 23:14:19,438 | train_utils.py:142 | Best Loss: 9.230368898322149e-06
INFO logger 2022-10-24 23:14:19,638 | server.py:191 | [Global round 2] Aggregating local models...
INFO logger 2022-10-24 23:14:20,447 | client_manager.py:88 | Parameter c=1.0. Sampled 3 client(s): ['LesCorts', 'PobleSec', 'ElBorn']
INFO logger 2022-10-24 23:14:22,213 | train_utils.py:142 | Best Loss: 1.3335206044997138e-05
INFO logger 2022-10-24 23:14:28,674 | train_utils.py:142 | Best Loss: 1.575978745992376e-05
INFO logger 2022-10-24 23:14:30,376 | train_utils.py:142 | Best Loss: 4.586990992144956e-06
INFO logger 2022-10-24 23:14:30,523 | server.py:191 | [Global round 3] Aggregat

In [30]:
global_model.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[-4.4958e-03,  1.0610e-01, -1.8101e-02],
                        [ 5.7758e-02, -5.1433e-02,  3.1065e-02],
                        [-1.0231e-01,  4.6284e-02, -4.5668e-02],
                        [-4.7838e-02, -2.3001e-02, -6.6809e-02],
                        [-2.9614e-02,  3.1829e-03, -2.6900e-02],
                        [-4.6026e-02,  5.3621e-03,  1.1022e-01],
                        [ 2.0108e-02,  3.4129e-02,  3.8213e-02],
                        [ 3.6525e-02, -1.5622e-02, -4.7462e-02],
                        [-7.4194e-02, -9.0040e-02,  3.7335e-03],
                        [-2.1357e-02, -6.1253e-02,  7.0864e-02],
                        [ 3.0791e-02,  4.4030e-03,  7.7984e-02],
                        [-5.6613e-02, -4.2900e-02,  4.3043e-02],
                        [ 1.3947e-01, -7.9259e-03, -1.8361e-02],
                        [ 4.2853e-02, -1.1919e-01, -1.1483e-03],
                        [-5.9175e-02, -1.5662e-01,  3.6776e-

In [31]:
history


History (client, train losses):
	LesCorts: {1: 1.8008789670259003e-05, 2: 1.6396968634961564e-05, 3: 1.5599051503317623e-05, 4: 1.4920589018571305e-05, 5: 1.586564123002507e-05, 6: 1.5312826370296365e-05, 7: 1.441099888071158e-05, 8: 1.5922363826216178e-05, 9: 1.5065540786286705e-05, 10: 1.4378315717992738e-05}
	ElBorn: {1: 3.354552038008783e-05, 2: 4.391456407370351e-05, 3: 3.96067809787204e-05, 4: 4.340028045115531e-05, 5: 3.519058868711069e-05, 6: 4.67073586059561e-05, 7: 2.604395969713999e-05, 8: 2.1004041088143427e-05, 9: 2.0255671428780323e-05, 10: 2.072920206351162e-05}
	PobleSec: {1: 1.9173772909213737e-05, 2: 1.7683451218310213e-05, 3: 1.7286858881080606e-05, 4: 1.6543134981253575e-05, 5: 1.6228538764907827e-05, 6: 1.5949743838569775e-05, 7: 1.578928327759439e-05, 8: 1.574607851212912e-05, 9: 1.5673683545355572e-05, 10: 1.5712797405366677e-05}

History (client, train metrics):
	LesCorts: {1: {'MSE': 0.0023051207, 'RMSE': 0.04801167249099282, 'MAE': 0.027620634, 'R^2': 0.64636